# CAPSTONE PROJECT: SUSHI RESTAURANT IN MILAN_ WEEK5

## Table of Content: 
#### 1.Introduction & Business Understanding
#### 2.Data
#### 3.Methodology
#### 4.Results
#### 5.Discussion & Conclusion

## INTRODUCTION & BUSINESS UNDERSTANDING

Milan is probably the most international city in Italy and for few years one of the most trendy restaurant type is Sushi.
For this reason the idea behind this problem is: if an investor wants to open a new Sushi restaurant, where would be the optimal location/neighborhood?

In such a problem many variables could play a vital role;
then, in this notebook the key assumptios are: 1) We are considering 88 NIL (Nuclei di Identità Locale) which are the statistica neighborhood of Milan; 2) The locations (Latitude & Longitude) are going to be retrived from Geopy 3) Total Spending Power is the KPI we are interested as a proxy of the overall wealth of the NIL; 4) we will use Foursquare API to register key locations for each NIL; 5) To compare every NIL and for the clustering processes we are going to use the % of Sushi Restaurants on the overall restaurants


## DATA

For all the above assumption, I will deep dive regarding data sources, needs & missing information:
1. The municipal entity of Milan offer an amazing portal ([Open Data Milan](http://dati.comune.milano.it/)). From this, it was possible to retrive the list of all the existing Neighborhoods. I discovered that this new classification is relatevely new and substituted the older Municipal Districts. Thus, I believe that this notebook could be a first analysis exploting this classification;
2. The geografical Information were missing and couldn't be extrapolated somewhere. For this reason, I used Geopy library to retreive Latitude & Longitude for each Neighborhood
3. Total Spending Power: this KPI was calculated considering the total numeber of Family per NIL & Consumption per Family for Grocery & Other Goods. The data were retrived again from ([Open Data Milan](http://dati.comune.milano.it/)), respectevely from ([Consumptions](http://dati.comune.milano.it/dataset/ds115-economia-spesa-media-mese-categoria-area-residenza-2007-2013)) & ([Families](http://dati.comune.milano.it/dataset/ds136-popolazione-famiglie-tipologia-municipio))
4. During this Course, I created a developer Account at Fourquare which I used to retrive all the necessary information. Following in this notebook, I cancelled all the lines which included my account info for Privacy Issues
5. Also for this KPI, I used all the info retrived from Fourquare and I indicated all the necessary steps in this Notebook


## Methodology

let's download libraries that we will be using

In [2]:
import geopandas as gpd
import geopy as gpy
from geopy.geocoders import Nominatim

!conda install -c conda-forge folium=0.5.0 --yes
!pip install geopy
import pandas as pd
import numpy as np
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


### Load the data

In [3]:
import pandas as pd
df = pd.read_excel('nil_adddress.xlsx', index_col=0) 
df.head()

,Neighborhood,Municipio,Address,Totale Abitanti,Famiglie,Comp. Fam,Total Spending Power
NIL,,,,,,,
1,Duomo,1,"Duomo,Milan,Italy",16889,8439,2.001303,309795690
2,Brera,1,"Brera,Milan,Italy",18501,9315,1.986151,341953650
3,Giardini Porta Venezia,1,"Giardini Porta Venezia,Milan,Italy",39,23,1.695652,844330
4,Guastalla,1,"Guastalla,Milan,Italy",15176,7584,2.001055,278408640
5,Vigentina,1,"Vigentina,Milan,Italy",13716,6536,2.098531,239936560


Before doing anything, let's explore how dataset

In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88 entries, 1 to 88
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Neighborhood          88 non-null     object 
 1   Municipio             88 non-null     int64  
 2   Address               88 non-null     object 
 3   Totale Abitanti       88 non-null     int64  
 4   Famiglie              88 non-null     int64  
 5   Comp. Fam             88 non-null     float64
 6   Total Spending Power  88 non-null     int64  
 7   location              88 non-null     object 
 8   point                 88 non-null     object 
 9   latitude              88 non-null     float64
 10  longitude             88 non-null     float64
 11  altitude              88 non-null     float64
dtypes: float64(4), int64(4), object(4)
memory usage: 11.4+ KB


In [102]:
df.describe()

,Municipio,Totale Abitanti,Famiglie,Comp. Fam,Total Spending Power,latitude,longitude,altitude
count,88.000000,88.000000,88.000000,88.000000,8.800000e+01,88.000000,88.000000,88.0
mean,5.295455,15855.386364,7902.636364,2.533498,2.901058e+08,45.470978,9.174152,0.0
std,2.609263,12944.493916,6546.487671,4.150605,2.403216e+08,0.031461,0.047447,0.0
min,1.000000,2.000000,2.000000,1.000000,7.342000e+04,45.401867,9.056185,0.0
25%,3.000000,4252.500000,2087.750000,1.970755,7.664130e+07,45.450038,9.141482,0.0
50%,5.000000,14750.000000,7432.000000,2.023845,2.728287e+08,45.472564,9.175946,0.0
75%,8.000000,23135.750000,11537.000000,2.083899,4.235233e+08,45.495764,9.208850,0.0
max,9.000000,62438.000000,31230.000000,40.761905,1.146453e+09,45.527369,9.262690,0.0


### Geocode Neighborhood Information

In [4]:
locator = Nominatim(user_agent="myGeocoder")
location = locator.geocode("Piazza Duomo 1, Milan, Italy")
print("Latitude ={}, Longitude = {}".format(location.latitude, location.longitude))

Latitude =43.875353700000005, Longitude = 8.014855860928806


In [6]:
from geopy.extra.rate_limiter import RateLimiter

geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
df['location'] = df['Address'].apply(geocode)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

df

,Neighborhood,Municipio,Address,Totale Abitanti,Famiglie,Comp. Fam,Total Spending Power,location,point,latitude,longitude,altitude
NIL,,,,,,,,,,,,
1,Duomo,1,"Duomo,Milan,Italy",16889,8439,2.001303,309795690,"(Duomo, Piazza del Duomo, Duomo, Municipio 1, ...","(45.4641385, 9.1885548, 0.0)",45.464138,9.188555,0.0
2,Brera,1,"Brera,Milan,Italy",18501,9315,1.986151,341953650,"(Brera, Municipio 1, Milano, Lombardia, 20121,...","(45.4715187, 9.1877351, 0.0)",45.471519,9.187735,0.0
3,Giardini Porta Venezia,1,"Giardini Porta Venezia,Milan,Italy",39,23,1.695652,844330,"(Giardini Porta Venezia, Porta Venezia, Milano...","(45.4747265, 9.20074954407837, 0.0)",45.474727,9.200750,0.0
4,Guastalla,1,"Guastalla,Milan,Italy",15176,7584,2.001055,278408640,"(Guastalla, Municipio 1, Milano, Lombardia, It...","(45.4582521, 9.2000227, 0.0)",45.458252,9.200023,0.0
5,Vigentina,1,"Vigentina,Milan,Italy",13716,6536,2.098531,239936560,"(Vigentina, Municipio 5, Milano, Lombardia, It...","(45.451087, 9.191564185415396, 0.0)",45.451087,9.191564,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
84,Parco Nord,9,"Parco Nord,Milan,Italy",98,43,2.279070,1578530,"(Parco Nord, Municipio 9, Milano, Lombardia, I...","(45.51994475, 9.181313083892288, 0.0)",45.519945,9.181313,0.0
85,Parco delle Abbazie,5,"Parco delle Abbazie,Milan,Italy",446,169,2.639053,6203990,"(Parco delle Abbazie, Municipio 5, Chiaravalle...","(45.41033475, 9.225145968161465, 0.0)",45.410335,9.225146,0.0
86,Parco dei Navigli,6,"Parco dei Navigli,Milan,Italy",297,143,2.076923,5249530,"(Parco dei Navigli, Municipio 6, Milano, Lomba...","(45.4227739, 9.140945883218809, 0.0)",45.422774,9.140946,0.0


In [7]:
import folium as folium 
map1 = folium.Map(
        location=[45.464138,9.188555],
        tiles='cartodbpositron',
        zoom_start=12,
)
df.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)
map1

In [104]:
#Let's add something more trendy
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

folium_map = folium.Map(
        location=[45.464138,9.188555],
        tiles='CartoDB dark_matter',
        zoom_start=12,
)

FastMarkerCluster(data=list(zip(df['latitude'].values, df['longitude'].values))).add_to(folium_map)
folium.LayerControl().add_to(folium_map)
folium_map

#### Define Foursquare Credentials and Version (obscured for privacy issues)

In [97]:
CLIENT_ID = 'XXXX' # your Foursquare ID
CLIENT_SECRET = 'XXXX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XXXX
CLIENT_SECRET:XXXX


#### Let's explore the first neighborhood in our dataframe

In [9]:
df.loc[1, 'Neighborhood']

'Duomo'

Get the neighborhood's latitude and longitude values.

In [10]:
neighborhood_latitude = df.loc[1, 'latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[1, 'longitude'] # neighborhood longitude value

neighborhood_name = df.loc[1, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Duomo are 45.4641385, 9.1885548.


Now, let's get the top 100 venues that are a radius of 500 meters.
First, let's create the GET request URL. Name your URL url.

In [98]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=XXXX&client_secret=XXXX&v=20180605&ll=45.4641385,9.1885548&radius=500&limit=100'

In [13]:
# Send the GET request and examine the results
results = requests.get(url).json()

In [14]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [15]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\Users\ricca\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Piazza del Duomo,Plaza,45.464190,9.189527
1,Galleria Vittorio Emanuele II,Monument / Landmark,45.465577,9.190024
2,Park Hyatt Milan,Hotel,45.465532,9.188911
3,Terrazze del Duomo,Scenic Lookout,45.464207,9.191075
4,Room Mate Giulia Hotel,Hotel,45.465250,9.189396


In [16]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

75 venues were returned by Foursquare.


In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
milan_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['latitude'],
                                   longitudes=df['longitude']
                                 )

Duomo
Brera
Giardini Porta Venezia
Guastalla
Vigentina
Ticinese
Magenta - S. Vittore
Parco Sempione
Garibaldi Repubblica
Centrale
Isola
Maciachini - Maggiolina
Greco
Niguarda - Cà Granda
Bicocca
Viale Monza
Adriano
Parco Lambro - Cimiano
Padova
Loreto
Buenos Aires - Venezia
Città Studi
Lambrate
Parco Forlanini - Ortica
Corsica
XXII Marzo
Porta Romana
Umbria - Molise
Ortomercato
Mecenate
Parco Monlué - Ponte Lambro
Triulzo Superiore
Rogoredo
Chiaravalle
Lodi - Corvetto
Scalo Romana
Ex OM - Morivione
Ripamonti
Quintosole
Ronchetto delle Rane
Gratosoglio - Ticinello
Stadera
Tibaldi
Navigli
S. Cristoforo
Barona
Cantalupa
Ronchetto sul Naviglio
Giambellino
Tortona
Washington
Bande Nere
Lorenteggio
Muggiano
Baggio
Forze Armate
Selinunte
De Angeli - Monte Rosa
Tre Torri
S. Siro
Quarto Cagnino
Quinto Romano
Figino
Trenno
Gallaratese
QT 8
Portello
Pagano
Sarpi
Ghisolfa
Villapizzone
Maggiore - Musocco
Cascina Triulza - Expo
Sacco
Stephenson
Quarto Oggiaro
Bovisa
Farini
Dergano
Affori
Bovisasca
C

In [19]:
print(milan_venues.shape)
milan_venues.head()

(2427, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Duomo,45.464138,9.188555,Piazza del Duomo,45.464190,9.189527,Plaza
1,Duomo,45.464138,9.188555,Galleria Vittorio Emanuele II,45.465577,9.190024,Monument / Landmark
2,Duomo,45.464138,9.188555,Park Hyatt Milan,45.465532,9.188911,Hotel
3,Duomo,45.464138,9.188555,Terrazze del Duomo,45.464207,9.191075,Scenic Lookout
4,Duomo,45.464138,9.188555,Room Mate Giulia Hotel,45.465250,9.189396,Hotel


In [20]:
#Let's group the total number of items we have in our dataset to have a better understanding of what we are dealing with.
milan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adriano,7,7,7,7,7,7
Affori,16,16,16,16,16,16
Baggio,8,8,8,8,8,8
Bande Nere,32,32,32,32,32,32
Barona,5,5,5,5,5,5
...,...,...,...,...,...,...
Viale Monza,31,31,31,31,31,31
Vigentina,48,48,48,48,48,48
Villapizzone,11,11,11,11,11,11


In [21]:
print('There are {} uniques categories.'.format(len(milan_venues['Venue Category'].unique())))

There are 266 uniques categories.


### One hot encoding: we turn categorical values in to zeros and ones to better accomodate Machine Learning algorithms.

In [22]:
# one hot encoding
milan_onehot = pd.get_dummies(milan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
milan_onehot['Neighborhood'] = milan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [milan_onehot.columns[-1]] + list(milan_onehot.columns[:-1])
milan_onehot = milan_onehot[fixed_columns]

milan_onehot.head()

,Yoga Studio,Abruzzo Restaurant,Accessories Store,Adult Education Center,African Restaurant,Airport,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,...,Turkish Restaurant,Tuscan Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volleyball Court,Whisky Bar,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
milan_onehot.shape

(2427, 266)

In [26]:
milan_grouped = milan_onehot.groupby('Neighborhood').mean().reset_index()
milan_grouped.shape
milan_grouped.head()

,Neighborhood,Yoga Studio,Abruzzo Restaurant,Accessories Store,Adult Education Center,African Restaurant,Airport,American Restaurant,Arcade,Argentinian Restaurant,...,Turkish Restaurant,Tuscan Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volleyball Court,Whisky Bar,Wine Bar,Wine Shop,Women's Store
0,Adriano,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Affori,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Baggio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bande Nere,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Barona,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
#Calculating frequencies of each category appearing in each neighbourhood.
milan_grouped = milan_onehot.groupby('Neighborhood').mean().reset_index()
milan_grouped

,Neighborhood,Yoga Studio,Abruzzo Restaurant,Accessories Store,Adult Education Center,African Restaurant,Airport,American Restaurant,Arcade,Argentinian Restaurant,...,Turkish Restaurant,Tuscan Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volleyball Court,Whisky Bar,Wine Bar,Wine Shop,Women's Store
0,Adriano,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,Affori,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,Baggio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,Bande Nere,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,Barona,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,Viale Monza,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
80,Vigentina,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.083333,0.0,0.0
81,Villapizzone,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
82,Washington,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [29]:
#Scaling down categories to columns that include word "Restaurant"
milan_restaurants_grouped = milan_grouped.loc[:, milan_grouped.columns.str.contains('Restaurant')]
milan_restaurants_grouped.head()

,Abruzzo Restaurant,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Brazilian Restaurant,Campanian Restaurant,Chinese Restaurant,Dim Sum Restaurant,...,Spanish Restaurant,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Tuscan Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
#Adding back Neighborhood column and resetting index so that we can manipulate its data.
milan_restaurants_grouped['Neighborhood'] = milan_grouped['Neighborhood'] 
milan_restaurants_grouped.set_index('Neighborhood', inplace = True)
milan_restaurants_grouped.reset_index(inplace=True)
milan_restaurants_grouped.head()

C:\Users\ricca\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Neighborhood,Abruzzo Restaurant,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Brazilian Restaurant,Campanian Restaurant,Chinese Restaurant,...,Spanish Restaurant,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Tuscan Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Adriano,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Affori,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Baggio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bande Nere,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Barona,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Finding the percentage of Sushi restaurants among other restaurant categories

In [32]:
#Checking the shape of the new dataframe.
milan_restaurants_grouped.shape

(84, 57)

In [33]:
#Summing up all the rows first.
milan_restaurants_grouped['Sum'] = milan_restaurants_grouped.sum(axis=1)
milan_restaurants_grouped.head()   

C:\Users\ricca\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Neighborhood,Abruzzo Restaurant,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Brazilian Restaurant,Campanian Restaurant,Chinese Restaurant,...,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Tuscan Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Sum
0,Adriano,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.428571
1,Affori,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125000
2,Baggio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.250000
3,Bande Nere,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.187500
4,Barona,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [57]:
#Finding the ratios of fast food restaurants by dividing the frequency of row sums by fast food restaurant restaurant frequencies of each row.
milan_restaurants_grouped['Ratio'] = milan_restaurants_grouped.loc[:,"Sushi Restaurant"].div(milan_restaurants_grouped['Sum'], axis=0)
milan_restaurants_grouped.head()

C:\Users\ricca\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Neighborhood,Abruzzo Restaurant,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Brazilian Restaurant,Campanian Restaurant,Chinese Restaurant,...,Szechuan Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Tuscan Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Sum,Ratio
0,Adriano,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.428571,0.0
1,Affori,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125000,0.0
2,Baggio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.250000,0.0
3,Bande Nere,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.187500,0.0
4,Barona,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,NaN


In order to have a better understanding of restaurants around each Neighborhood, we can call top 10 restaurant categories by their frequencies

In [58]:
#Filling not available values with zeros, so that we don't have any problems while calling top restaurant categories from Foursquare API.
milan_restaurants_grouped['Ratio'] = milan_restaurants_grouped['Ratio'].fillna(0)
milan_restaurants_grouped.head()

C:\Users\ricca\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Neighborhood,Abruzzo Restaurant,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Brazilian Restaurant,Campanian Restaurant,Chinese Restaurant,...,Szechuan Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Tuscan Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Sum,Ratio
0,Adriano,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.428571,0.0
1,Affori,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125000,0.0
2,Baggio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.250000,0.0
3,Bande Nere,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.187500,0.0
4,Barona,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [59]:
milan_restaurants_grouped_na = milan_restaurants_grouped.drop(['Sum','Ratio'], axis=1)
milan_restaurants_grouped_na.head()

,Neighborhood,Abruzzo Restaurant,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Brazilian Restaurant,Campanian Restaurant,Chinese Restaurant,...,Spanish Restaurant,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Tuscan Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Adriano,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Affori,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Baggio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bande Nere,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Barona,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Final Data Preparation**

In [60]:
#To avoid confusion, I make a new dataset that includes Neighborhood and Ratio columns. 
milan = milan_restaurants_grouped[['Neighborhood','Ratio']]
milan.head()

,Neighborhood,Ratio
0,Adriano,0.0
1,Affori,0.0
2,Baggio,0.0
3,Bande Nere,0.0
4,Barona,0.0


In [61]:
#Then I join Spending Power, Latitude and Longtitude of Neighborhoods.
milan_ff = pd.merge(milan, df, on='Neighborhood')
milan_ff.head()

,Neighborhood,Ratio,Municipio,Address,Totale Abitanti,Famiglie,Comp. Fam,Total Spending Power,location,point,latitude,longitude,altitude
0,Adriano,0.0,2,"Adriano,Milan,Italy",17508,7739,2.262308,284098690,"(Adriano, Crescenzago, Milano, Lombardia, Ital...","(45.5135717, 9.251201850969608, 0.0)",45.513572,9.251202,0.0
1,Affori,0.0,9,"Affori,Milan,Italy",25341,12259,2.067134,450027890,"(Affori, Milano, Lombardia, 20161, Italia, (45...","(45.5170295, 9.1696533, 0.0)",45.517029,9.169653,0.0
2,Baggio,0.0,7,"Baggio,Milan,Italy",30017,15110,1.986565,554688100,"(Baggio, Milano, Lombardia, 20153, Italia, (45...","(45.4613839, 9.089843, 0.0)",45.461384,9.089843,0.0
3,Bande Nere,0.0,5,"Bande Nere,Milan,Italy",44576,22767,1.957922,835776570,"(Bande Nere, Piazzale Giovanni delle Bande Ner...","(45.461504, 9.1364845, 0.0)",45.461504,9.136484,0.0
4,Barona,0.0,6,"Barona,Milan,Italy",16988,8042,2.112410,295221820,"(Barona, Quartiere Sant'Ambrogio, Milano, Lomb...","(45.4307236, 9.153586498645357, 0.0)",45.430724,9.153586,0.0


In [109]:
milan_sushi=milan_ff[['Neighborhood','Total Spending Power','Ratio','latitude','longitude']]
milan_sushi.head()

,Neighborhood,Total Spending Power,Ratio,latitude,longitude
0,Adriano,284098690,0.0,45.513572,9.251202
1,Affori,450027890,0.0,45.517029,9.169653
2,Baggio,554688100,0.0,45.461384,9.089843
3,Bande Nere,835776570,0.0,45.461504,9.136484
4,Barona,295221820,0.0,45.430724,9.153586


In [110]:
#To totally rinse the dataframe from Non-Sushi Related items, I drop out all the 0 values under Ratio column.
milan_sushi_scope = milan_sushi[milan_sushi.Ratio != 0]
milan_sushi_scope

,Neighborhood,Total Spending Power,Ratio,latitude,longitude
5,Bicocca,149886930,0.166667,45.514917,9.211138
8,Brera,341953650,0.038462,45.471519,9.187735
10,Buenos Aires - Venezia,1146453300,0.035714,45.477892,9.212902
13,Centrale,345734780,0.028571,45.484352,9.203372
17,Corsica,359354190,0.125000,45.463909,9.230802
18,De Angeli - Monte Rosa,398854150,0.125000,45.476130,9.147302
26,Garibaldi Repubblica,105614670,0.058824,45.483527,9.189933
29,Giardini Porta Venezia,844330,0.142857,45.474727,9.200750
32,Guastalla,278408640,0.062500,45.458252,9.200023
39,Magenta - S. Vittore,317504790,0.058824,45.464689,9.169665


In [111]:
#Let's turn frequency ratio to percentage.
milan_sushi_scope['Percentage']=milan_sushi_scope['Ratio']*100
milan_sushi_scope.head()

C:\Users\ricca\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Neighborhood,Total Spending Power,Ratio,latitude,longitude,Percentage
5,Bicocca,149886930,0.166667,45.514917,9.211138,16.666667
8,Brera,341953650,0.038462,45.471519,9.187735,3.846154
10,Buenos Aires - Venezia,1146453300,0.035714,45.477892,9.212902,3.571429
13,Centrale,345734780,0.028571,45.484352,9.203372,2.857143
17,Corsica,359354190,0.125000,45.463909,9.230802,12.500000


In [112]:
#Putting Total Spending Power and Percentage to the end of the table.
milan_sushi_scope=milan_sushi_scope[['Neighborhood','latitude','longitude','Total Spending Power','Percentage']]
milan_sushi_scope.head()

,Neighborhood,latitude,longitude,Total Spending Power,Percentage
5,Bicocca,45.514917,9.211138,149886930,16.666667
8,Brera,45.471519,9.187735,341953650,3.846154
10,Buenos Aires - Venezia,45.477892,9.212902,1146453300,3.571429
13,Centrale,45.484352,9.203372,345734780,2.857143
17,Corsica,45.463909,9.230802,359354190,12.500000


### Modelling

In [86]:
# set number of clusters
kclusters = 5

milan_sushi_scope_clustering = milan_sushi_scope.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(milan_sushi_scope_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 0, 2, 0, 0, 0, 3, 4, 0, 0])

In [87]:
milan_sushi_scope.head()

,Cluster Labels,Neighborhood,latitude,longitude,Total Spending Power,Percentage
5,3,Bicocca,45.514917,9.211138,149886930,16.666667
8,0,Brera,45.471519,9.187735,341953650,3.846154
10,2,Buenos Aires - Venezia,45.477892,9.212902,1146453300,3.571429
13,0,Centrale,45.484352,9.203372,345734780,2.857143
17,0,Corsica,45.463909,9.230802,359354190,12.500000


## Results

First thing; I will explore the results via a Map to have a better understanding. Then I'll deep dive in each Cluster

In [89]:
# create map
map_clusters = folium.Map(location=[45.464138,9.188555], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(milan_fastfood_scope['latitude'], milan_fastfood_scope['longitude'], milan_fastfood_scope['Neighborhood'], milan_fastfood_scope['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## CLUSTER 0

In [90]:
cluster_0 = milan_fastfood_scope.loc[milan_fastfood_scope['Cluster Labels'] == 0, milan_fastfood_scope.columns[[1,4] + list(range(5, milan_fastfood_scope.shape[1]))]]
cluster_0 = cluster_0.sort_values(by=['Total Spending Power'], ascending=False)
cluster_0

,Neighborhood,Total Spending Power,Percentage
78,Umbria - Molise,424844830,8.333333
18,De Angeli - Monte Rosa,398854150,12.500000
73,Ticinese,368458270,8.695652
17,Corsica,359354190,12.500000
13,Centrale,345734780,2.857143
8,Brera,341953650,3.846154
43,Navigli,320882110,3.225806
39,Magenta - S. Vittore,317504790,5.882353
54,Porta Romana,313870500,2.857143
47,Pagano,313797080,18.181818


This first cluster is characterized by Medium-Low Spending Power & Medium % of Sushi Restaurant

## CLUSTER 1

In [91]:
cluster_1 = milan_fastfood_scope.loc[milan_fastfood_scope['Cluster Labels'] == 1, milan_fastfood_scope.columns[[1,4] + list(range(5, milan_fastfood_scope.shape[1]))]]
cluster_1 = cluster_1.sort_values(by=['Total Spending Power'], ascending=False)
cluster_1

,Neighborhood,Total Spending Power,Percentage
44,Niguarda - Cà Granda,685338990,100.000000
83,XXII Marzo,588424590,5.555556
69,Selinunte,503330810,16.666667


This second cluster is characterized by Medium-High Spending Power & Medium- High % of Sushi Restaurant

## CLUSTER 2

In [92]:
cluster_2 = milan_fastfood_scope.loc[milan_fastfood_scope['Cluster Labels'] == 2, milan_fastfood_scope.columns[[1,4] + list(range(5, milan_fastfood_scope.shape[1]))]]
cluster_2 = cluster_2.sort_values(by=['Total Spending Power'], ascending=False)
cluster_2

,Neighborhood,Total Spending Power,Percentage
10,Buenos Aires - Venezia,1146453300,3.571429


This third cluster is characterized by High Spending Power & Low % of Sushi Restaurant.
This Case is particulary interesting: due to its High Spending Power (it is the max among the 88 NIL), this Neighborhood represents itself a cluster

## CLUSTER 3

In [93]:
cluster_3 = milan_fastfood_scope.loc[milan_fastfood_scope['Cluster Labels'] == 3, milan_fastfood_scope.columns[[1,4] + list(range(5, milan_fastfood_scope.shape[1]))]]
cluster_3 = cluster_3.sort_values(by=['Total Spending Power'], ascending=False)
cluster_3

,Neighborhood,Total Spending Power,Percentage
72,Tibaldi,228740010,7.142857
55,Portello,157449190,12.500000
5,Bicocca,149886930,16.666667
26,Garibaldi Repubblica,105614670,5.882353


This cluster is characterized by Low Spending Power & Medium % of Sushi Restaurant

## CLUSTER 4

In [94]:
cluster_4 = milan_fastfood_scope.loc[milan_fastfood_scope['Cluster Labels'] == 4, milan_fastfood_scope.columns[[1,4] + list(range(5, milan_fastfood_scope.shape[1]))]]
cluster_4 = cluster_4.sort_values(by=['Total Spending Power'], ascending=False)
cluster_4

,Neighborhood,Total Spending Power,Percentage
75,Tre Torri,36599870,30.000000
29,Giardini Porta Venezia,844330,14.285714


This cluster is characterized by Low Spending Power & Medium-High % of Sushi Restaurant

## Discussion & Conclusion

So we arrived at the end of this interesting (and fun) analysis. In this section, my goal is twofold: to further discuss the overall assumption of the model and some further development points; to wrap up and arrive at some kind of conclusion.

1. In this analysis, the overall assumption is that Total Spending Power is a proxy of the wealth of the people living in the Neighborhood and thus, the higher the better. As it is a Proxy, it could be improved: some examples that came to my mind (if info are available): total spending in restaurants per capita, average usage of restaurants, addition information regarding the type of families etc.

2. Let's recap the identified clusters:
    a) **CLUSTER 0**: Medium-Low Spending Power & Medium %; **CLUSTER 1**: Medium-High Spending Power & Medium-High %; **CLUSTER 2**: High Spending Power & Low %; **CLUSTER 3**: Low Spending Power & Medium %; **CLUSTER 4**: Low Spending Power & Medium-High %;
    In this case, we can make the assumption that it is necessary to have at least Medium in both Spending Power & % of Sushi Restaurants. The first part it is straighforward: the more the Spending power, the more likely people in that hood are going out to restaurants etc. Instead, the second part embodies the fact that if there are already some sushi restaurants in the area, this has a positive impact because people have already tested this type of cousine and thus it is more likely that are going again (from a business point of view, this assumptions replicate the logics behind "followers" strategy rather than "first-movers"). 
    
    Thus, narrowing down the analysis, it is possible to exclude Cluster 0, 2,3,4 and we can identify Cluster 1 as the best fit
    
Concluding, this analysis led to the identification of a Cluster which represents a first group of areas where to open a Sushi Restaurant. From this analysis is then necessary to deep dive in this areas to better understand the context, real estate prices, consumers habits etc. which are out of scope for this analysis
